# **1️⃣ Imports**

In [202]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

import joblib


# **2️⃣ Load Dataset**

In [203]:
df = pd.read_csv("../../datasets/diabetes_dataset.csv")
df.head()

,year,gender,age,location,race:AfricanAmerican,race:Asian,race:Caucasian,race:Hispanic,race:Other,hypertension,heart_disease,smoking_history,bmi,hbA1c_level,blood_glucose_level,diabetes
0,2020,Female,32.0,Alabama,0,0,0,0,1,0,0,never,27.32,5.0,100,0
1,2015,Female,29.0,Alabama,0,1,0,0,0,0,0,never,19.95,5.0,90,0
2,2015,Male,18.0,Alabama,0,0,0,0,1,0,0,never,23.76,4.8,160,0
3,2015,Male,41.0,Alabama,0,0,1,0,0,0,0,never,27.32,4.0,159,0
4,2016,Female,52.0,Alabama,1,0,0,0,0,0,0,never,23.75,6.5,90,0


# **3️⃣ Data Cleaning (Based on EDA)**

 **Replace invalid zeros with NaN**

In [204]:
df.columns

Index(['year', 'gender', 'age', 'location', 'race:AfricanAmerican',
       'race:Asian', 'race:Caucasian', 'race:Hispanic', 'race:Other',
       'hypertension', 'heart_disease', 'smoking_history', 'bmi',
       'hbA1c_level', 'blood_glucose_level', 'diabetes'],
      dtype='str')

In [205]:
cols_to_fix = ['bmi', 'hbA1c_level', 'blood_glucose_level']

df[cols_to_fix] = df[cols_to_fix].replace(0, np.nan)


**Fill missing values with median**

In [206]:
# for col in invalid_zero_cols:
#     df[col].fillna(df[col].median(), inplace=True)

In [207]:
df.isnull().sum()

year                    0
gender                  0
age                     0
location                0
race:AfricanAmerican    0
race:Asian              0
race:Caucasian          0
race:Hispanic           0
race:Other              0
hypertension            0
heart_disease           0
smoking_history         0
bmi                     0
hbA1c_level             0
blood_glucose_level     0
diabetes                0
dtype: int64

In [208]:
# numeric_cols = df.select_dtypes(include=['int64','float64']).columns

# df[numeric_cols] = df[numeric_cols].fillna(
#     df[numeric_cols].median()
# )


In [209]:
df.isnull().sum()


year                    0
gender                  0
age                     0
location                0
race:AfricanAmerican    0
race:Asian              0
race:Caucasian          0
race:Hispanic           0
race:Other              0
hypertension            0
heart_disease           0
smoking_history         0
bmi                     0
hbA1c_level             0
blood_glucose_level     0
diabetes                0
dtype: int64

# **4️⃣ Feature / Target Split**

In [210]:
X = df.drop(["diabetes", "year"], axis=1)
y = df["diabetes"]


# **5️⃣ Train–Test Split**

In [211]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [212]:
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'string']).columns


print("Numeric:", numeric_cols)
print("Categorical:", categorical_cols)


Numeric: Index(['age', 'race:AfricanAmerican', 'race:Asian', 'race:Caucasian',
       'race:Hispanic', 'race:Other', 'hypertension', 'heart_disease', 'bmi',
       'hbA1c_level', 'blood_glucose_level'],
      dtype='str')
Categorical: Index(['gender', 'location', 'smoking_history'], dtype='str')


In [213]:
df['diabetes'].value_counts(normalize=True)


diabetes
0    0.915
1    0.085
Name: proportion, dtype: float64

In [214]:
df['diabetes'].unique()



array([0, 1])

In [215]:
df['diabetes'].value_counts()


diabetes
0    91500
1     8500
Name: count, dtype: int64

In [216]:
df.shape


(100000, 16)

In [217]:
df.head()

,year,gender,age,location,race:AfricanAmerican,race:Asian,race:Caucasian,race:Hispanic,race:Other,hypertension,heart_disease,smoking_history,bmi,hbA1c_level,blood_glucose_level,diabetes
0,2020,Female,32.0,Alabama,0,0,0,0,1,0,0,never,27.32,5.0,100,0
1,2015,Female,29.0,Alabama,0,1,0,0,0,0,0,never,19.95,5.0,90,0
2,2015,Male,18.0,Alabama,0,0,0,0,1,0,0,never,23.76,4.8,160,0
3,2015,Male,41.0,Alabama,0,0,1,0,0,0,0,never,27.32,4.0,159,0
4,2016,Female,52.0,Alabama,1,0,0,0,0,0,0,never,23.75,6.5,90,0


# **6️⃣ Train Multiple Models**

### **Logistic Regression**

In [218]:
# Numeric Pipeline

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])


In [219]:
# Categorical Pipeline

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])


In [220]:
# Combine Using ColumnTransformer

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_cols),
    ("cat", categorical_transformer, categorical_cols)
])


In [221]:
lr_pipeline = Pipeline([
    ("preprocessing", preprocessor),
    ("model", LogisticRegression(
        max_iter=1000,
        class_weight="balanced"
    ))
])


In [222]:
lr_pipeline.fit(X_train, y_train)


lr_preds = lr_pipeline.predict(X_test)

print("Logistic Regression Accuracy:", accuracy_score(y_test, lr_preds))
print("\nClassification Report:\n", classification_report(y_test, lr_preds))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, lr_preds))


Logistic Regression Accuracy: 0.8862

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.89      0.93     18300
           1       0.42      0.87      0.57      1700

    accuracy                           0.89     20000
   macro avg       0.70      0.88      0.75     20000
weighted avg       0.94      0.89      0.90     20000


Confusion Matrix:
 [[16237  2063]
 [  213  1487]]


In [223]:
# joblib.dump(lr_pipeline, "../../models/diabetes_logistic_pipeline.pkl")


### **Random Forest**

In [224]:
rf_pipeline = Pipeline([
    ("preprocessing", preprocessor),  # reuse your ColumnTransformer
    ("model", RandomForestClassifier(
        n_estimators=200,
        random_state=42,
        class_weight="balanced"
    ))
])



In [225]:
rf_pipeline.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessing', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers c

In [226]:
rf_preds = rf_pipeline.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))
print("\nClassification Report:\n", classification_report(y_test, rf_preds))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, rf_preds))


Random Forest Accuracy: 0.9723

Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.99     18300
           1       0.99      0.68      0.81      1700

    accuracy                           0.97     20000
   macro avg       0.98      0.84      0.90     20000
weighted avg       0.97      0.97      0.97     20000


Confusion Matrix:
 [[18283    17]
 [  537  1163]]


In [227]:
# joblib.dump(rf_pipeline, "../../models/diabetes_rf_pipeline.pkl")


# **SVM**

In [228]:
svm_pipeline = Pipeline([
    ("preprocessing", preprocessor),
    ("model", SVC(
        kernel="rbf",
        C=1.0,
        gamma="scale",
        probability=True,
        random_state=42
    ))
])


In [229]:
svm_pipeline.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessing', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers c

In [230]:
svm_preds = svm_pipeline.predict(X_test)

print("SVM Accuracy:", accuracy_score(y_test, svm_preds))
print("\nClassification Report:\n", classification_report(y_test, svm_preds))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, svm_preds))


SVM Accuracy: 0.96375

Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98     18300
           1       0.97      0.59      0.74      1700

    accuracy                           0.96     20000
   macro avg       0.96      0.80      0.86     20000
weighted avg       0.96      0.96      0.96     20000


Confusion Matrix:
 [[18264    36]
 [  689  1011]]


In [231]:
# joblib.dump(svm_pipeline, "../../models/diabetes_svm_pipeline.pkl")


In [232]:
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

scale_ratio = 91500 / 8500  # imbalance ratio

xgb_pipeline = Pipeline([
    ("preprocessing", preprocessor),
    ("model", XGBClassifier(
        objective="binary:logistic",
        eval_metric="logloss",
        scale_pos_weight=scale_ratio,
        random_state=42,
        n_estimators=300,
        max_depth=5,
        learning_rate=0.1
    ))
])

xgb_pipeline.fit(X_train, y_train)

xgb_preds = xgb_pipeline.predict(X_test)

print("XGBoost Accuracy:", accuracy_score(y_test, xgb_preds))
print("\nClassification Report:\n", classification_report(y_test, xgb_preds))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, xgb_preds))

XGBoost Accuracy: 0.91635

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.92      0.95     18300
           1       0.50      0.90      0.65      1700

    accuracy                           0.92     20000
   macro avg       0.75      0.91      0.80     20000
weighted avg       0.95      0.92      0.93     20000


Confusion Matrix:
 [[16803  1497]
 [  176  1524]]


In [233]:
probs = xgb_pipeline.predict_proba(X_test)[:, 1]

In [234]:

threshold = 0.4
custom_preds = (probs > threshold).astype(int)

print(classification_report(y_test, custom_preds))
print(confusion_matrix(y_test, custom_preds))

              precision    recall  f1-score   support

           0       0.99      0.89      0.94     18300
           1       0.43      0.93      0.59      1700

    accuracy                           0.89     20000
   macro avg       0.71      0.91      0.76     20000
weighted avg       0.94      0.89      0.91     20000

[[16226  2074]
 [  121  1579]]


In [235]:
from sklearn.metrics import classification_report

for t in [0.5, 0.45, 0.4, 0.35, 0.3]:
    preds = (probs > t).astype(int)
    print(f"\nThreshold: {t}")
    print(classification_report(y_test, preds))


Threshold: 0.5
              precision    recall  f1-score   support

           0       0.99      0.92      0.95     18300
           1       0.50      0.90      0.65      1700

    accuracy                           0.92     20000
   macro avg       0.75      0.91      0.80     20000
weighted avg       0.95      0.92      0.93     20000


Threshold: 0.45
              precision    recall  f1-score   support

           0       0.99      0.90      0.95     18300
           1       0.47      0.91      0.62      1700

    accuracy                           0.90     20000
   macro avg       0.73      0.91      0.78     20000
weighted avg       0.95      0.90      0.92     20000


Threshold: 0.4
              precision    recall  f1-score   support

           0       0.99      0.89      0.94     18300
           1       0.43      0.93      0.59      1700

    accuracy                           0.89     20000
   macro avg       0.71      0.91      0.76     20000
weighted avg       0.94  

In [253]:
joblib.dump({
    "model": xgb_pipeline,
    "threshold": 0.45
}, "../../models/diabetes_model.pkl")

['../../models/diabetes_model.pkl']

In [254]:
# lr_loaded = joblib.load("../../models/diabetes_logistic_pipeline.pkl")
# rf_loaded = joblib.load("../../models/diabetes_rf_pipeline.pkl")
# svm_loaded = joblib.load("../../models/diabetes_svm_pipeline.pkl")

# sample_df = X_test.iloc[[0]]

# print("LR Prediction:", lr_loaded.predict(sample_df)[0])
# print("RF Prediction:", rf_loaded.predict(sample_df)[0])
# print("SVM Prediction:", svm_loaded.predict(sample_df)[0])


In [255]:
# # Build Pipeline with SMOTE
# rf_pipeline = ImbPipeline([
#     ("imputer", SimpleImputer(strategy="median")),
#     ("smote", SMOTE(random_state=42)),
#     ("model", RandomForestClassifier(random_state=42))
# ])

In [256]:
# Hyperparameter Space
# param_dist = {
#     "model__n_estimators": [200, 300, 400, 500],
#     "model__max_depth": [None, 10, 20, 30],
#     "model__min_samples_split": [2, 5, 10],
#     "model__min_samples_leaf": [1, 2, 4],
#     "model__max_features": ["sqrt", "log2"]
# }

In [257]:
# rf_search = RandomizedSearchCV(
#     rf_pipeline,
#     param_distributions=param_dist,
#     n_iter=25,
#     cv=5,
#     scoring="recall",  # 🔥 Optimize for diabetic detection
#     n_jobs=-1,
#     random_state=42
# )

In [258]:
# rf_search.fit(X_train, y_train)

In [259]:
# print("Best Parameters:", rf_search.best_params_)
# print("Best CV Recall:", rf_search.best_score_)

In [260]:
# # Evaluate Best Model
# best_rf = rf_search.best_estimator_

# preds = best_rf.predict(X_test)
# probs = best_rf.predict_proba(X_test)[:, 1]

# print("\nImproved Random Forest Accuracy:", accuracy_score(y_test, preds))
# print("\nClassification Report:\n", classification_report(y_test, preds))
# print("\nConfusion Matrix:\n", confusion_matrix(y_test, preds))
# print("\nROC-AUC:", roc_auc_score(y_test, probs))

In [261]:
# Save Best Model
# joblib.dump(best_rf, "../../models/diabetes_best_rf_pipeline.pkl")

<!-- “I optimized for recall to reduce false negatives in diabetic detection. This slightly increased false positives, reducing accuracy marginally, but improved sensitivity — which is more critical in medical screening systems.” -->

### "Build XGBoost Pipeline"

In [262]:
# xgb_pipeline = ImbPipeline([
#     ("imputer", SimpleImputer(strategy="median")),
#     ("smote", SMOTE(random_state=42)),
#     ("model", XGBClassifier(
#         objective="binary:logistic",
#         eval_metric="logloss",
#         use_label_encoder=False,
#         random_state=42
#     ))
# ])


### "Hyperparameter Tuning"

In [263]:
# param_dist = {
#     "model__n_estimators": [200, 300, 400],
#     "model__max_depth": [3, 5, 7],
#     "model__learning_rate": [0.01, 0.05, 0.1],
#     "model__subsample": [0.8, 1.0],
#     "model__colsample_bytree": [0.8, 1.0],
#     "model__gamma": [0, 1, 5]
# }

# xgb_search = RandomizedSearchCV(
#     xgb_pipeline,
#     param_distributions=param_dist,
#     n_iter=20,
#     cv=5,
#     scoring="recall",  # prioritize detecting diabetes
#     n_jobs=-1,
#     random_state=42
# )

# xgb_search.fit(X_train, y_train)

# print("Best Parameters:", xgb_search.best_params_)
# print("Best CV Recall:", xgb_search.best_score_)


Evaluate Best XGBoost Model

In [264]:
# best_xgb = xgb_search.best_estimator_

# preds = best_xgb.predict(X_test)
# probs = best_xgb.predict_proba(X_test)[:, 1]

# print("XGBoost Accuracy:", accuracy_score(y_test, preds))
# print("\nClassification Report:\n", classification_report(y_test, preds))
# print("\nConfusion Matrix:\n", confusion_matrix(y_test, preds))
# print("\nROC-AUC:", roc_auc_score(y_test, probs))


In [265]:
# XGBoost reduced false negatives from:

# 24 (original RF)

# 17 (tuned RF)

# 🔥 10 (XGBoost)

# That is a major improvement.

# You are now detecting:

# 44 out of 54 diabetic patients
# (~81% sensitivity)

# For a medical screening model, that is strong.



In [266]:
# XGBoost increased:

# False Positives (28 vs 24 vs 16 earlier)

# But that is acceptable in medical screening.

# Why?

# Because:

# It is safer to wrongly flag a healthy person
# than to miss a sick patient.

# This is the correct medical bias

 <!-- ### "“XGBoost achieved the highest recall for diabetic cases, reducing false negatives significantly while maintaining stable ROC-AUC. Since medical screening prioritizes sensitivity over raw accuracy, XGBoost was selected as the final deployed model.”" -->

In [267]:
# probs = best_xgb.predict_proba(X_test)[:, 1]

# custom_preds = (probs > 0.40).astype(int)

# print(classification_report(y_test, custom_preds))


In [268]:
# For academic “balanced metrics”:
# → Deploying XGBoost default threshold

# “I adjusted the classification threshold to increase sensitivity for diabetic detection. This reduced false negatives further, which is critical in medical screening, even though overall accuracy slightly decreased.”

In [269]:
# joblib.dump(best_xgb, "../../models/diabetes_final_model.pkl")
